In [10]:
import pandas as pd
import matplotlib.pyplot as plt
from pyfmi import load_fmu
import pyfmi
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyfmi import load_fmu
import pyfmi
import numpy as np
import joblib
from tensorflow import keras
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import mean_squared_error
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [40]:
## 灰箱子模型

## 计算COP数值
# 讲模型此刻的Tcon, Teva,以及计算出来的，注意单位是摄氏度

def Copcal(PLR,TEVA,TCON):
    # Load the saved model
    pre_model = keras.models.load_model("NanWang_COP_model.h5")
    # Load the scaler object from the file
    scaler = joblib.load('NanWang_cop.pkl')

    data = {
        'Number_1': [TCON],
        'Number_2': [TEVA],
        'Number_3': [PLR],
            }

    new_df = pd.DataFrame(data)
    X_other_standardized = scaler.transform(new_df)
    prediction = pre_model.predict(X_other_standardized)
    Cop_value = prediction[0][0]
    
    return Cop_value

## 计算总能耗

# 确定冷机的功率和额定容量，以及水泵的额定功率 额定功率97kW, 额定容量 517 kW

def Energycal(M_room,Q_pre,q_pre,T_room,N_chiler,Cop_value): 
    t_pre = M_room * (T_room - 24)*1.004 /(N_chiler * 546) + 15
    ## 备注： 15为估测的系统启动需要的延时值
    t_m = (Q_pre - q_pre*(60 + t_pre ))/ (N_chiler * 546)
    t_stay = t_pre - t_m
    E_chiller = N_chiler * 97 * t_m +  q_pre/Cop_value * t_stay
    E_pump = N_chiler * 6 * t_m + N_chiler * 6 * t_stay * (q_pre//N_chiler)**3
    E_total = E_chiller + E_pump
    if t_pre > 120:
        E_total = 20000000
    return E_total, t_pre


## 冷机台数参数寻优
## 该项目中目前估算出的M_room值大约为：4188
def Optichi(M_room,Q_pre, q_pre, T_room, TEVA,TCON):
    min_E_total = float('inf')  # Initialize min_E_total to positive infinity
    best_N_chiller = None
    

    for N_chiller in np.arange(1, 3, 1):
        PLR = q_pre/N_chiller/3517
        if PLR > 1.2:
            PLR = 1.2
        elif PLR < 0.2:
            PLR = 0.2
        Cop_value = Copcal(PLR,TEVA,TCON)
        
        E_total, t_pre = Energycal(M_room,Q_pre, q_pre, T_room, N_chiller, Cop_value)
        
        # Check if current E_total is less than the minimum found so far
        if E_total < min_E_total:
            min_E_total = E_total
            best_N_chiller = N_chiller
            best_t_m = t_pre

    return best_N_chiller, best_t_m

In [ ]:
#  请先构建预测模型：
# 参考： 2024.8.21 南网项目预测模型搭建

In [14]:
## 预测模型

def predictcool(average_Tdb,week_number,history1,history2,history3):
    # Load the model for prediction
    clf = lgb.Booster(model_file='simulation_nanwang_total.txt')
    scaler = joblib.load('scaler_nanwang_total.pkl')
    # build the new dataframe
    data = {
        'Number_1': [average_Tdb],
        'Number_2': [week_number],
        'Number_3': [history1],
        'Number_4': [history2],
        'Number_5': [history3]
    }

    new_df = pd.DataFrame(data)
    # standardize the new dataframe
    X_predcit = scaler.transform(new_df)
    prediction = clf.predict(X_predcit)
    value = prediction[0]
    return value

def predictsigle(average_Tdb,week_number,history1,history2,history3):
    # Load the model for prediction
    clf = lgb.Booster(model_file='simulation_nanwang_sigle.txt')
    scaler = joblib.load('scaler_nanwang_sigle.pkl')
    # build the new dataframe
    data = {
        'Number_1': [average_Tdb],
        'Number_2': [week_number],
        'Number_3': [history1],
        'Number_4': [history2],
        'Number_5': [history3]
    }

    new_df = pd.DataFrame(data)
    # standardize the new dataframe
    X_predcit = scaler.transform(new_df)
    prediction = clf.predict(X_predcit)
    value = prediction[0]
    return value

In [41]:
## 优化模型测试
Optichi(4188,68384, 360.219122, 25.9, 18.95,34.61)

(1, 29.63192087912087)

周三模型预测以及优化结果分析

In [18]:
predictcool(27 ,4,78796.62943,87001.792793,81403)

70894.93325596255

In [25]:
predictsigle(26 ,4,423 ,416,413)

334.09493551039225

In [31]:
## 优化模型测试
Optichi(4188/2,70894.93325596255, 334.09493551039225, 27, 11,30)

(1, 22.70101098901099)

In [ ]:
2024-08-27	87001.792793	275.855000	5.256493
2024-08-28	81298.577104	219.981667	6.159496
2024-08-29	78796.629439	247.928333	5.297003

In [ ]:
1292    413.275940
1293    416.376730
1294    423.498600

In [3]:
102792 - 70894

31898

In [4]:
31898/70894

0.44993934606595765

In [8]:
38/334

0.11377245508982035

In [21]:
## 优化模型测试
Optichi(4188/2,70894, 334.09, 27, 11,30)

(1, 38.103032967032966)

In [23]:
## 优化模型测试
Optichi(4188,72796.62, 439.37, 27, 11,30)

(1, 38.103032967032966)